In [1]:
import numpy as np
import pandas as pd
from misc import load, load_class_labels
from sklearn.metrics import classification_report

predictions_path = "predictions/"
test_info = load("../dataset/labels/test.csv")
class_labels = load_class_labels('../dataset/classes.txt')

def load_predictions(path, model_name):
    predictions = np.load(path + model_name + ".npy")
    names = pd.read_csv(path + model_name + "_names.csv")['filename'].values
    indices = np.argsort(names)
    return predictions[indices], names[indices]

In [2]:
from scipy.stats import entropy
from scipy.special import softmax

vit_predictions, names = load_predictions(predictions_path, "ViT_extended_degraded")
en_predictions, _ = load_predictions(predictions_path, "efficientnet_v2_extended_degraded")

test_names = test_info['filename'].values
test_labels = test_info['label'].values

ordered_indices = np.argsort(test_names)
ordered_labels = test_labels[ordered_indices]


final_prob = np.zeros(vit_predictions.shape)
for i, (vit_pred, en_pred) in enumerate(zip(vit_predictions, en_predictions)):
    vit_entropy = 1 / entropy(vit_pred)
    en_entropy = 1 / entropy(en_pred)
    
    normalized_vit_entropy = vit_entropy / (vit_entropy + en_entropy)
    normalized_en_entropy = en_entropy / (en_entropy + vit_entropy)
    
    final_prob[i] = normalized_vit_entropy * vit_pred + normalized_en_entropy * en_pred
    print(final_prob[i].sum())
    
final_predictions = np.argmax(final_prob, axis=1)

0.9999999392018708
1.0000001366715878
0.9999999689630386
0.9999999937463144
0.9999998670259629
1.000000072450348
1.0000000253385224
1.0000001805865395
0.9999999314350134
0.999999929266778
1.0000000174622983
0.9999999851097527
0.9999999400261004
1.0000001242273129
1.0000000702120815
0.9999999753945303
1.0000000864165486
1.0000000233103492
0.9999998646284212
0.9999999822648533
0.9999998607978569
0.9999999933945674
0.9999999408100848
1.0000000236623237
0.9999999798474164
1.0000000651126015
1.0000000271766112
1.0000000079708116
1.0000000957297743
0.99999985708655
0.9999999775645847
0.9999999807520226
1.0000000222789822
1.0000000593590812
0.9999999750198185
1.0000000605417654
1.000000003608875
1.0000000497202564
0.9999999774731805
0.9999999681240297
1.0000000346717002
0.9999999518749831
0.999999995528924
0.9999999596352609
0.9999999202002527
1.0000000352774805
1.0000000400818863
1.0000000593518052
0.9999999586661943
0.9999998906796463
1.000000043736236
1.0000000011971792
0.999999919671609
1

In [3]:
from tensorflow import keras


print(classification_report(ordered_labels, final_predictions, target_names=class_labels))

2024-01-07 15:18:40.352531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 15:18:40.352595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 15:18:40.557656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 15:18:40.949023: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 15:18:42.929885: W tensorflow/compiler/tf2

                               precision    recall  f1-score   support

                      macaron       0.83      0.91      0.87        55
                      beignet       0.71      0.84      0.77        61
                      cruller       0.71      0.64      0.67        53
                  cockle_food       0.60      0.63      0.62        51
                       samosa       0.54      0.90      0.68        41
                     tiramisu       0.62      0.58      0.60        52
                      tostada       0.64      0.53      0.58        57
                     moussaka       0.41      0.56      0.47        54
                     dumpling       0.58      0.57      0.58        49
                      sashimi       0.62      0.74      0.67        46
                        knish       0.72      0.44      0.55        48
                    croquette       0.45      0.64      0.53        47
                     couscous       0.29      0.71      0.41        45
     

/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gabriele/PycharmProjects/Visual/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [61]:
def top_k(predictions, gt, k = 1):
    ohe = keras.utils.to_categorical(gt, num_classes=251)
    m = keras.metrics.TopKCategoricalAccuracy(k)
    m.update_state(ohe, predictions)
    return m.result().numpy()

print(top_k(final_prob, ordered_labels, 3))

0.7723862
